In [9]:
!pip install -U pip
!pip install transformers

In [10]:
from datasets import load_dataset

conll2003 = load_dataset("conll2003")
conll2003

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [11]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

classifier = pipeline("ner", model=model, tokenizer=tokenizer)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


In [12]:
example = "My name is Wolfgang and I live in Berlin"

ner_results = classifier(example)
print(ner_results)

[{'entity': 'B-PER', 'score': np.float32(0.9990139), 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': np.float32(0.999645), 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


In [13]:
X_train = conll2003["train"][:5]
len(X_train["tokens"])

5

In [14]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

y_true = [['B-PER', 'I-PER', 'O'], ['B-LOC', 'O']]
y_pred = [['B-PER', 'I-PER', 'O'], ['B-LOC', 'B-LOC']]

print("Precision:", precision_score(y_true, y_pred))
print("Recall:", recall_score(y_true, y_pred))
print("F1 Score:", f1_score(y_true, y_pred))
print(classification_report(y_true, y_pred))

Precision: 0.6666666666666666
Recall: 1.0
F1 Score: 0.8
              precision    recall  f1-score   support

         LOC       0.50      1.00      0.67         1
         PER       1.00      1.00      1.00         1

   micro avg       0.67      1.00      0.80         2
   macro avg       0.75      1.00      0.83         2
weighted avg       0.75      1.00      0.83         2



In [15]:
import torch
import torch.nn.functional as F


def to_str(tokens: list) -> str:
    is_first = True
    result = ""

    for token in tokens:
        if not is_first and token not in (".", "'s"):
            result += " "
        is_first = False
        result += token
        
    return result


#batch = tokenizer(X_train["tokens"], is_split_into_words=True, return_tensors="pt")

inp = list(map(to_str, X_train["tokens"]))
t = tokenizer(inp, padding=True, return_tensors="pt")



# classifier([["I live in London"], ["I live in London"]])

with torch.no_grad():
    model(**t)
#     outputs = model(**batch)
#     print(outputs)

RuntimeError: Placeholder storage has not been allocated on MPS device!

In [106]:
import numpy as np


a = np.random.rand(6).reshape(2,3)

a, np.argmax(a, axis=0), np.argmax(a, axis=-1)

(array([[0.58398942, 0.43840728, 0.9057182 ],
        [0.76069001, 0.04714852, 0.94553024]]),
 array([1, 0, 1]),
 array([1, 0, 1]))

# HF Trainer Tutorial

To understand lib better
https://huggingface.co/docs/transformers/en/training

In [1]:
# Settings

num_train_records = 10
num_labels = 5


In [2]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("yelp_review_full")
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [4]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(num_train_records))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(num_train_records))

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=num_labels, torch_dtype="auto")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [7]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    return metric.compute(predictions=predictions, references=labels)

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer", 
    eval_strategy="epoch",
    )

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [14]:
trainer.train()

  0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 1.4167077541351318, 'eval_accuracy': 0.38, 'eval_runtime': 27.0449, 'eval_samples_per_second': 3.698, 'eval_steps_per_second': 0.481, 'epoch': 1.0}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 1.580071210861206, 'eval_accuracy': 0.29, 'eval_runtime': 13.0221, 'eval_samples_per_second': 7.679, 'eval_steps_per_second': 0.998, 'epoch': 2.0}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 1.3865406513214111, 'eval_accuracy': 0.46, 'eval_runtime': 10.6694, 'eval_samples_per_second': 9.373, 'eval_steps_per_second': 1.218, 'epoch': 3.0}
{'train_runtime': 97.2908, 'train_samples_per_second': 3.084, 'train_steps_per_second': 0.401, 'train_loss': 1.059646215194311, 'epoch': 3.0}


TrainOutput(global_step=39, training_loss=1.059646215194311, metrics={'train_runtime': 97.2908, 'train_samples_per_second': 3.084, 'train_steps_per_second': 0.401, 'total_flos': 78935442739200.0, 'train_loss': 1.059646215194311, 'epoch': 3.0})

In [13]:
trainer.predict(small_eval_dataset)

  0%|          | 0/13 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[-0.02541004, -0.04039362,  0.38818917,  0.28195083, -0.1499093 ],
       [-0.14094451, -0.09143201,  0.4345184 ,  0.4684406 , -0.05507018],
       [ 0.15295082,  0.35771677,  0.02276882,  0.10684581, -0.6638198 ],
       [-0.29001212, -0.43959072,  0.3632385 ,  0.54328114,  0.46342796],
       [-0.3094497 , -0.370084  ,  0.33227748,  0.5673134 ,  0.65000093],
       [-0.29842785, -0.8053097 ,  0.2793649 ,  0.32033736,  1.0743855 ],
       [ 0.03125766,  0.01347399,  0.36804235,  0.22924419, -0.2562423 ],
       [-0.06575987, -0.01651558,  0.21632692,  0.26552007, -0.20219022],
       [ 0.04994113,  0.07351896,  0.17684965,  0.39502186, -0.30381703],
       [-0.27694228, -0.37256166,  0.27082598,  0.51420444,  0.47650757],
       [ 0.13963957,  0.36850053,  0.03155496,  0.16136107, -0.7942679 ],
       [-0.01249037,  0.63185894, -0.42213073,  0.19529338, -0.9898524 ],
       [ 0.03379242,  0.4727533 , -0.09952519,  0.12683341, -1.0153639 ],
       [ 